#### joint_trajectory_controller
- 3 different types
    1. `position_controllers/JointTrajectoryController`
    2. `velocity_controllers/JointTrajectoryController`
    3. `effort_controllers/JointTrajectoryController`
- it exposes an action interface (`control_msgs/FollowJointTrajectoryAction`) and a topic interface (`trajectory_msgs/JointTrajectory`)
- `control_msgs/FollowJointTrajectoryAction`
- `trajectory_msgs/JointTrajectory`
    - it provides an array of `trajectory_msgs/JointTrajectoryPoint`
        - `trajectory_msgs/JointTrajectoryPoint` has also arrays for positions, velocities, accelerations, and efforts.
            - one for each joint
            - are they 2-way directional?
            
#### Sawyer with Gazebo
- installation instructions [here](https://sdk.rethinkrobotics.com/intera/Gazebo_Tutorial)
    - you might need to
        - `sudo apt install ros-noetic-image-proc`
- on Ubuntu 20.04 (as of 09/10/22) OpenCV 4.0 gives a compilation error
    - at `src/sawyer_simulator/sawyer_gazebo/src/head_interface.cpp:71`
        - use `cv::IMREAD_UNCHANGED` instead
- it provides its own implementation of `ros_control` by a Gazebo plugin
    - `libsawyer_gazebo_ros_control.so`
    - all `EffortJointInterface` for joint (or transmission)
        - but it seems to load position controllers and velocity controllers too
            - I guess it depends on Sawyer's implementation of `ros_control`
- do hardware interfaces expose ROS topics?
    - I guess not
    - you can manually send a comman to these topic
        - `rostopic pub -1 /robot/right_joint_position_controller/joints/right_j0_controller/command std_msgs/Float64 "data: 1.0"`
- many error messages related with nodelets
- `rosservice call /robot/controller_manager/list_controllers` gives a list of controllers
    - what does the `sawyer_sim_controllers/SawyerGravityController` do?
    - many controllers share same resources?

#### Move Group Python interface
- The Sawyer robot in Gazebo lives in the namespace of `/robot`
    - give that namespace when a commander is created
        - `robot = moveit_commander.RobotCommander(ns='/robot')`


#### MoveIt! with a Sawyer in Gazebo or in real
- is it true that the MoveIt! only needs `control_msgs/FollowJointTrajectoryActionGoal` action server to control robots.
- would it be convenient to create an `PlanningSceneInterface` with `synchronous=True`


In [ ]:
import sys

import rospy
import moveit_commander


moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node("move_group_python_interface_tutorial", anonymous=True)

robot = moveit_commander.RobotCommander(ns="/robot")
scene = moveit_commander.PlanningSceneInterface(synchronous=True)

group_name = "right_arm"
move_group = moveit_commander.MoveGroupCommander(group_name)

# Pose goal
pose_goal = geometry_msgs.msg.Pose()
pose_goal.orientation.x = 0.1869791285289449
pose_goal.orientation.y = 0.9520766806140123
pose_goal.orientation.z = -0.18284561015180614
pose_goal.orientation.w = 0.15860732194250726

pose_goal.position.x = 0.53
pose_goal.position.y = 0.1
pose_goal.position.z = 0.2

move_group.set_pose_target(pose_goal)

# `go()` returns a boolean indicating whether the planning and execution was successful.
success = move_group.go(wait=True)
# Calling `stop()` ensures that there is no residual movement
move_group.stop()
# It is always good to clear your targets after planning with poses.
# Note: there is no equivalent function for clear_joint_value_targets().
move_group.clear_pose_targets()

#### MoveIt! collision objects
- to simulate a gripper (or something attached to a robot) you can use a collision object
    - it might be a better option to include those objects into the URDF file of the robot if it is under your control
- you can simply use `scene.add_box` or `scene.add_mesh`
    - these use `scene.add_object` underneath
- you can also directly use `scene.add_object`
    - then you need to create an `moveit_msgs.msg.CollisionObject` instance yourself
    - you need to use `shape_msgs.msg.SolidPrimitive` and `geometry_msgs.msg.Pose` messages

In [ ]:
from geometry_msgs.msg import PoseStamped
from moveit_msgs.msg import CollisionObject
from shape_msgs.msg import SolidPrimitive


pose = PoseStamped()
pose.header.frame_id = "base"
pose.pose.orientation.w = 1
pose.pose.position.x = 0.56
pose.pose.position.z = -0.28

scene.add_box("Table", pose, (0.78, 1.56, 0.20))

# if you want to model the gripper in detail
scene.add_mesh("mesh", pose, stl_file_path)

# if you want to make your own CollisionObject
# add a CollisionObject to the scene
col = CollisionObject()
col.id = "gripper"
col.header.frame_id = "right_hand"

primitive = SolidPrimitive()
primitive.type = primitive.CYLINDER
primitive.dimensions = [0.2, 0.1]

col.primitives = [primitive]
col.primitive_poses = [pose]

scene.add_object(col)

In [ ]:
# attach the gripper base
pose = PoseStamped()
pose.header.frame_id = "right_hand"
pose.pose.orientation.w = 1
pose.pose.position.z = 0.04

scene.add_box("gripper_base", pose, (0.22, 0.08, 0.08))
touch_links = robot.get_link_names("right_arm")
scene.attach_box("right_hand", "gripper_base", touch_links=touch_links)

# attach the left finger
pose = PoseStamped()
pose.header.frame_id = "right_hand"
pose.pose.orientation.w = 1
pose.pose.position.x = -0.08
pose.pose.position.z = 0.12

scene.add_box("gripper_finger_l", pose, (0.04, 0.04, 0.08))
touch_links = robot.get_link_names("right_arm") + ["gripper_base"]
scene.attach_box("right_hand", "gripper_finger_l", touch_links=touch_links)

# attach the right finger
pose = PoseStamped()
pose.header.frame_id = "right_hand"
pose.pose.orientation.w = 1
pose.pose.position.x = 0.08
pose.pose.position.z = 0.12

scene.add_box("gripper_finger_r", pose, (0.04, 0.04, 0.08))
touch_links = robot.get_link_names("right_arm") + ["gripper_base"]
scene.attach_box("right_hand", "gripper_finger_r", touch_links=touch_links)


#### MoveIt! attaching objects
- once objects are added to the scene you can attach those to a link
- it would be simpler to use `scene.attach_box` or `scene.attach_mesh` than using `scene.attach_object`
    - `attach_box` and `attach_mesh` use `scene.attach_object` underneath
- you can directly use `scene.attach_object`
    - then, you need to create an `moveit_msgs.msg.AttachedCollisionObject` instance from `moveit_msgs.msg.CollisionObject`


#### works with a physical Swayer
- edit `intera.sh` as
    - `robot_hostname="021708CP00025.local"`
    - `your_ip="192.168.."`
- startup procesure
    1. `./src/intera_sdk/intera.sh` in the root of the workspace
    2. enable the robot
        - `rosrun intera_interface enable_robot.py -e`
    3. start a trajectory action server
        - `rosrun intera_interface joint_trajectory_action_server.py`
    4. start a `moveit_config` for the sawyer
        - `roslaunch sawyer_moveit_config sawyer_moveit.launch`


#### joint target and pose target

In [ ]:
joint_goal = move_group.get_current_joint_values()
# joint_goal[0] = 0
# joint_goal[1] = -tau / 8
# joint_goal[2] = 0
# joint_goal[3] = -tau / 4
# joint_goal[4] = 0
# joint_goal[5] = tau / 6  # 1/6 of a turn
joint_goal[6] = 0

# The go command can be called with joint values, poses, or without any
# parameters if you have already set the pose or joint target for the group
move_group.go(joint_goal, wait=True)

# Calling ``stop()`` ensures that there is no residual movement
move_group.stop()

In [ ]:
import numpy as np


pose_goal = move_group.get_current_pose()
pose_goal.pose.position.x = 0.5
pose_goal.pose.position.y = 0.2
pose_goal.pose.position.z = 0.2
pose_goal.pose.orientation.w = np.cos(np.pi / 2)
pose_goal.pose.orientation.x = 0
pose_goal.pose.orientation.y = -1
pose_goal.pose.orientation.z = 0

move_group.set_pose_target(pose_goal)

# move_group.plan() will show the plan in rviz!
plan = move_group.plan()
success = move_group.execute(plan[1])

# `go()` returns a boolean indicating whether the planning and execution was successful.
success = move_group.go(wait=True)
# Calling `stop()` ensures that there is no residual movement
move_group.stop()
# It is always good to clear your targets after planning with poses.
# Note: there is no equivalent function for clear_joint_value_targets().
move_group.clear_pose_targets()

#### grasping
- z = -0.01 seems to be a good height to grab cubes on the desk


In [ ]:
from intera_interface import (
    SimpleClickSmartGripper,
    get_current_gripper_interface
)

gripper = get_current_gripper_interface()

# close
gripper.set_ee_signal_value('grip', False)

# open
gripper.set_ee_signal_value('grip', True)


#### after grab
- add the object to the scene
- attach the object to the grasping group
    - or to two finger objects

In [ ]:
pose_goal = move_group.get_current_pose()
pose_goal.pose.position.x = 0.5
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = -0.01
pose_goal.pose.orientation.w = np.cos(np.pi / 2)
pose_goal.pose.orientation.x = 0
pose_goal.pose.orientation.y = -1
pose_goal.pose.orientation.z = 0

move_group.set_pose_target(pose_goal)

# move_group.plan() will show the plan in rviz!
plan = move_group.plan()
success = move_group.execute(plan[1])
move_group.clear_pose_targets()


pose_goal = move_group.get_current_pose()
pose_goal.pose.position.x = 0.5
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = -0.01
pose_goal.pose.orientation.w = np.cos(np.pi / 2)
pose_goal.pose.orientation.x = 0
pose_goal.pose.orientation.y = -1
pose_goal.pose.orientation.z = 0

move_group.set_pose_target(pose_goal)

# move_group.plan() will show the plan in rviz!
plan = move_group.plan()
success = move_group.execute(plan[1])
move_group.clear_pose_targets()


pose_goal = move_group.get_current_pose()
pose_goal.pose.position.x = 0.5
pose_goal.pose.position.y = 0.0
pose_goal.pose.position.z = 0.2
pose_goal.pose.orientation.w = np.cos(np.pi / 2)
pose_goal.pose.orientation.x = 0
pose_goal.pose.orientation.y = -1
pose_goal.pose.orientation.z = 0

move_group.set_pose_target(pose_goal)

# move_group.plan() will show the plan in rviz!
plan = move_group.plan()
success = move_group.execute(plan[1])
move_group.clear_pose_targets()


pose_goal = move_group.get_current_pose()
pose_goal.pose.position.x = 0.5
pose_goal.pose.position.y = 0.0
pose_goal.pose.position.z = -0.01
pose_goal.pose.orientation.w = np.cos(np.pi / 2)
pose_goal.pose.orientation.x = 0
pose_goal.pose.orientation.y = -1
pose_goal.pose.orientation.z = 0

move_group.set_pose_target(pose_goal)
# move_group.plan() will show the plan in rviz!
plan = move_group.plan()
success = move_group.execute(plan[1])
move_group.clear_pose_targets()

In [ ]:
# pose_goal = move_group.get_current_pose()
# pose_goal.pose.position.x = 0.5
# pose_goal.pose.position.y = -0.2
# pose_goal.pose.position.z = -0.01
# pose_goal.pose.orientation.w = np.cos(np.pi / 2)
# pose_goal.pose.orientation.x = 0
# pose_goal.pose.orientation.y = -1
# pose_goal.pose.orientation.z = 0

# move_group.set_pose_target(pose_goal)

# # move_group.plan() will show the plan in rviz!
# plan = move_group.plan()
# success = move_group.execute(plan[1])
# move_group.clear_pose_targets()


# close
gripper.set_ee_signal_value('grip', False)
scene.attach_box("right_hand", "Box_1", touch_links=["gripper_finger_l", "gripper_finger_r"])


pose_goal = move_group.get_current_pose()
pose_goal.pose.position.x = 0.5
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = 0.1
pose_goal.pose.orientation.w = np.cos(np.pi / 2)
pose_goal.pose.orientation.x = 0
pose_goal.pose.orientation.y = -1
pose_goal.pose.orientation.z = 0

move_group.set_pose_target(pose_goal)

# move_group.plan() will show the plan in rviz!
plan = move_group.plan()
success = move_group.execute(plan[1])
move_group.clear_pose_targets()


pose_goal = move_group.get_current_pose()
pose_goal.pose.position.x = 0.5
pose_goal.pose.position.y = 0.0
pose_goal.pose.position.z = 0.08
pose_goal.pose.orientation.w = np.cos(np.pi / 2)
pose_goal.pose.orientation.x = 0
pose_goal.pose.orientation.y = -1
pose_goal.pose.orientation.z = 0

move_group.set_pose_target(pose_goal)

# move_group.plan() will show the plan in rviz!
plan = move_group.plan()
success = move_group.execute(plan[1])
move_group.clear_pose_targets()


gripper.set_ee_signal_value('grip', True)
scene.remove_attached_object("right_hand", name="Box_1")